# Uplift Models for Promoters Campaigns

### Summary

During the Promoters Campaigns, the effect on monthly cashflow of sending promoters to an outlet may vary in a range from very negative to very positive. A negative effect implies that money was spent in sending the promoters but actually the monthly sales diminished, which is not desirable for Tsel or for the outlet. So we want to send promoters only to outlets where we expect the outcome will be at least enough positive to justify the investment.

Uplift models can be used to target the best outlets to send the promoters. In this notebook, we will load the data from the Promoters Pilot to make an uplift model. The model can be used to identify the best outlets to send promoters.

### Load packages

The model is based on the "causalml" library from Uber and the data processing is based on typical data science libraries.

In [ ]:
%matplotlib inline
from causalml.dataset import synthetic_data
import math
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score, classification_report, confusion_matrix, accuracy_score, roc_auc_score 
import statsmodels.api as sm
import seaborn as sns
import math as math
import matplotlib.pyplot as plt
import seaborn as sns

### Load Promoters Pilot table and organice it

This dataset contains the cashflows from february and march for the Promoters Pilot Test and Control Groups.

The Control Group was created by finding for each Test outlet, another random outlet having the same classification, type and region. This means both groups have the same size (but we will see later that we don't have features for all outlets, so the sizes will become different).

In [ ]:
# Load table
PP_experiment_df = pd.read_csv('Promoters Pilot input table v2.txt')
PP_experiment_df.rename(columns={'Outlet_id':'outlet_id'},inplace=True)

In [ ]:
# Assign target variable for uplift model
PP_experiment_df['Target_variable'] = PP_experiment_df['Cashflow_march_1_29'] # Two options: 'Cashflow_march_1_29' and 'Delta_feb_mar'
PP_experiment_df.head()

In [ ]:
# List of test and control outlets
outlets_list = PP_experiment_df['outlet_id'].tolist()
len(outlets_list)

### Check stats of target variable

In [ ]:
test = PP_experiment_df.loc[PP_experiment_df['Treatment'] == 1]
test['Target_variable'].mean()

In [ ]:
control = PP_experiment_df.loc[PP_experiment_df['Treatment'] == 0]
control['Target_variable'].mean()

### Load outlets master table of features

This table is used as the input for the Reseller's Model, but we will use it to add features to the Test and Control groups outlets. The features will be useful during the modelling stage.

In [ ]:
master_df = pd.read_csv('../../../data/reseller/07_model_output/gridsearchcv/ra_mck_int_gridsearchcv_master_prepared.csv')

In [ ]:
#master_df.head()
len(master_df)
len(master_df.outlet_id.unique())
len(master_df.columns)

### Check how many Test and Control outlets in master table of features

Not at outlets in the Test and Control groups are present in the master table, so we will check how many there are. Note that this causes the Test and Control groups to have smaller and different sizes.

In [ ]:
# Make list of all outlets in master table of features
master_outlets_list = master_df.outlet_id.unique().tolist()
len(master_outlets_list) # Outlets in master table of features

In [ ]:
# Check how many Test and Control outlets in master table
Exp_outlets_in_master_df = PP_experiment_df[PP_experiment_df['outlet_id'].isin(master_outlets_list)]
len(Exp_outlets_in_master_df)

In [ ]:
# Check for only Test outlets
Exp_outlets_in_master_1_df = Exp_outlets_in_master_df.loc[Exp_outlets_in_master_df['Treatment'] == 1]
len(Exp_outlets_in_master_1_df)

In [ ]:
# Check for only Control outlets
Exp_outlets_in_master_0_df = Exp_outlets_in_master_df.loc[Exp_outlets_in_master_df['Treatment'] == 0]
len(Exp_outlets_in_master_0_df)

### Check target variable stats after filtering

We have to calculate stats again after filtering.

In [ ]:
# Check the target variable average of Test outlets
y_test = Exp_outlets_in_master_1_df.loc[Exp_outlets_in_master_1_df['Treatment'] == 1]
y_test['Target_variable'].mean()

In [ ]:
# Check the target variable average of Control outlets
y_control = Exp_outlets_in_master_0_df.loc[Exp_outlets_in_master_0_df['Treatment'] == 0]
y_control['Target_variable'].mean()

### Filter master table of features by outlets list (Test and Control)

Now we filter the master table of features by the Test and Control outlets.

In [ ]:
PP_features_df = master_df[master_df['outlet_id'].isin(outlets_list)]
len(PP_features_df)

In [ ]:
# Check number of columns
len(PP_features_df.columns) 

In [ ]:
# Clean NA columns just in case
PP_features_df = PP_features_df.dropna(axis=1)
len(PP_features_df.columns)

### Attach master table of features to Promoters Pilot table

Now both tables are joined to produce a Promoters Pilot master table.

In [ ]:
PP_master_df = PP_experiment_df.join(PP_features_df.set_index('outlet_id'), on='outlet_id')
#PP_master_df.head()

In [ ]:
PP_master_df = PP_master_df.dropna(axis=0)
len(PP_master_df)

In [ ]:
len(PP_master_df.columns)

### Define target variable, features and treatment

We will define the target variable, features and treatment. These will be used not just for the uplift models, but for two previous models we have to do to select main features and check sampling bias in the Test and Control groups.

#### Create target variable y

In [ ]:
y = PP_master_df['Target_variable']
len(y)

#### Create treatment

In [ ]:
treatment = PP_master_df['Treatment']
len(treatment)

#### Create features X

In [ ]:
# Encode categorical variables
from sklearn.preprocessing import LabelEncoder

labelencoder1= LabelEncoder() #initializing an object of class LabelEncoder
PP_master_df['Type'] = labelencoder1.fit_transform(PP_master_df['Type'])

labelencoder2= LabelEncoder() #initializing an object of class LabelEncoder
PP_master_df['Classification'] = labelencoder2.fit_transform(PP_master_df['Classification'])

labelencoder3= LabelEncoder() #initializing an object of class LabelEncoder
PP_master_df['Region'] = labelencoder3.fit_transform(PP_master_df['Region'])

PP_master_df.head()

In [ ]:
# Check for relevant columns (*please edit the indices)
PP_master_df.columns[0:20].tolist()

In [ ]:
# Relevant columns
df1 = PP_master_df.iloc[:,3:6]
df2 = PP_master_df.iloc[:,10:11]
df3 = PP_master_df.iloc[:,13:2109]
df4 = PP_master_df.iloc[:,2214:2372]
x = pd.concat([df1,df2,df3,df4], axis=1)
len(x.columns)

In [ ]:
# Keep only float and int columns
columns = []
for j in range(0,len(x.columns)):
    if x.dtypes[j] == 'float64' or x.dtypes[j] == 'int64':
        columns.append(x.columns[j])
x = x[columns]
print(len(x.columns)) # In the next section the number of features will be reduces to 50

### Random forest model to define subset of x features to use in uplift model

We create this random forest classifier model to select the top features to use in the uplift model. This will modify the number of features in dataframe "x" which we will use for the uplift model.

#### Create target variable for the model

In [ ]:
# Classify target variable in 4 categories corresponding to quartiles, and use categories as as target
# variable only for this model
PP_master_df['Target_variable'].describe()

In [ ]:
cash = PP_master_df['Target_variable'].tolist()
PP_master_df['target_class1'] = -1
target1 = PP_master_df['target_class1'].tolist()
for i in range(0,len(cash)):
    if cash[i] > np.percentile(PP_master_df.Target_variable, 75):
        target1[i] = 4
    elif cash[i] <= np.percentile(PP_master_df.Target_variable, 75) and cash[i] > np.percentile(PP_master_df.Target_variable, 50):
        target1[i] = 3
    elif cash[i] <= np.percentile(PP_master_df.Target_variable, 50) and cash[i] > np.percentile(PP_master_df.Target_variable, 25):  
        target1[i] = 2
    elif cash[i] <= np.percentile(PP_master_df.Target_variable, 25):
        target1[i] = 1
    else:
        print('Error')

In [ ]:
# Assign target variable
PP_master_df['target_class1'] = target1
y_class = PP_master_df['target_class1']
len(y_class)

#### Create model

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y_class, test_size=0.30, random_state=42)

In [ ]:
regressor = RandomForestClassifier()
regressor.fit(x_train,y_train)

In [ ]:
y_pred = regressor.predict(x_test)

#### Check model

In [ ]:
print(confusion_matrix(y_test,y_pred))

In [ ]:
print(classification_report(y_test,y_pred))

#### Chooose top variables

In [ ]:
Variables = pd.Series(x.columns)
Feature_importances = pd.Series(regressor.feature_importances_)
Feature_importances_dic = {'Variable': Variables, "Feature_importance": Feature_importances}
Feature_importances_df = pd.DataFrame(Feature_importances_dic)
fis = Feature_importances_df.sort_values(by="Feature_importance", ascending=False)
fis.iloc[0:50,:]

In [ ]:
vars_ranked = fis['Variable'].tolist()
top_vars = vars_ranked[0:50]
X = x[top_vars]

### Check for possible sample bias

For the uplift modelling to work properly, a good practice is to create Test and Control groups from random samples. Here we will check if there seems to be a sample bias. For this purpose, we will create a random forest classifier to try to predict the Treatment variable as the target variable. The prediction results must be low to guarantee that there is not an important bias.

In [ ]:
x_check_bias = x.drop(columns=['Promoter_days'])
x_train, x_test, t_train, t_test = train_test_split(x_check_bias, treatment, test_size=0.30, random_state=42)

In [ ]:
regressor = RandomForestClassifier()
regressor.fit(x_train,t_train)

In [ ]:
t_pred = regressor.predict(x_test)

In [ ]:
print(confusion_matrix(t_test,t_pred))

In [ ]:
print(classification_report(t_test,t_pred)) # High values might imply bias

In [ ]:
print(accuracy_score(t_test,t_pred))

In [ ]:
Variables = pd.Series(x.columns)
Feature_importances = pd.Series(regressor.feature_importances_)
Feature_importances_dic = {'Variable': Variables, "Feature_importance": Feature_importances}
Feature_importances_df = pd.DataFrame(Feature_importances_dic)
fis = Feature_importances_df.sort_values(by="Feature_importance", ascending=False)
fis.iloc[0:10,:]

### Uplift modeling

Now we'll proceed with creating an Uplift model for the target variable given the promoters presence or absence in the outlets.

We use a T-learner from the meta-learner based methods. A meta-learner is a framework to estimate the Conditional Average Treatment effect (CATE). The T-learner makes a model for the Test Group, then another model for the Control Group, and uses the differences (i.e. Test - Control) to estimate uplift.

#### Prepare the dataset

In [ ]:
from causalml.inference.meta import BaseTRegressor
from xgboost import XGBRegressor
from causalml.inference.meta import XGBTRegressor
from causalml.metrics import plot

# Define target variable
y = PP_master_df['Target_variable']

In [ ]:
data = pd.concat([
    pd.DataFrame({"y": y, "treatment": treatment}),
    pd.DataFrame(X)],
    axis = 1
)
data

#### Check target variable stats for Test and Control groups

In [ ]:
data_test = data.loc[data['treatment'] == 1]
data_test['y'].mean()

In [ ]:
data_test = data.loc[data['treatment'] == 0]
data_test['y'].mean()

#### The model

In [ ]:
X_train, X_test, y_train, y_test, treatment_train, treatment_test = train_test_split(X, y, treatment, test_size=0.30, random_state=42)

In [ ]:
## Training T-learner on train
learner_t = XGBTRegressor(learner=XGBRegressor(random_state=42))
learner_t.fit(X=X_train, treatment=treatment_train, y=y_train)

## Get predictions, on the test set
uplift, outcome_c, outcome_t = learner_t.predict(X=X_test, return_components=True)

In [ ]:
## Aggregating everything on a dataframe
df = pd.DataFrame({'y': y_test,
                   'w': treatment_test,
                   'T-Learner': uplift.reshape(-1)
                  })

#### Model evaluation

In [ ]:
# Qini plot (where uplift is in y-axis, which is average test - average control for for top p population)
plot(df,kind='qini', outcome_col='y', treatment_col='w',figsize=(10, 3.3))

In [ ]:
# Qini score
from causalml.metrics import auuc_score, qini_score
print('\nQINI Score\n',qini_score(df))

In [ ]:
# AUUC
print('AUUC:\n',auuc_score(df))

SHAP values can be used to understand the feature importances.

In [ ]:
# Feature importances using SHAP
import shap
from sklearn.ensemble import RandomForestRegressor

# Raw SHAP values
shap_values = learner_t.get_shap_values(X=X_test,
                                        tau=learner_t.predict(X_test),
                                        #we may specify the exact model to be used as additonal one
                                        model_tau_feature = RandomForestRegressor(n_estimators=100))

In [ ]:
len(shap_values.get(1))

The SHAP importance plot will show us the following:

- Feature importance: Variables are ranked in descending order.
- Impact: The horizontal location shows whether the effect of that value is associated with a higher or lower prediction.

In [ ]:
# SHAP importance plot
learner_t.plot_shap_values(X=X_test, tau=learner_t.predict(X_test))

In [ ]:
# The following table matches the SHAP importance plot to the feature names
feature_importance = pd.DataFrame({'Feature': X_test.columns,
                   'Number': range(0, len(X_test.columns))
                  })
feature_importance

### Prediction using the model

With the model ready, uplift predictions can be made over any set of outlets (even non-random sets). We will use the outlets from X_test as a didactic example.

In [ ]:
# Get predictions, on the test set
uplift, outcome_c, outcome_t = learner_t.predict(X=X_test, return_components=True)

In [ ]:
uplift

In [ ]:
outcome_t.get(1) - outcome_c.get(1) # This is equal to the uplift

In the histogram of the uplift, we can observe more than half of the outlets have a positive uplift.

In [ ]:
# Histogram of the uplift
plt.hist(uplift, bins=25)

However, the cost of sending a promoter is nearly 3 million IDR + any Tsel admin cost. We will then look for outlets with more than 4 million IDR.

In [ ]:
promoters_threshold = 4000000
selected_outlets = uplift[uplift>promoters_threshold]
len(selected_outlets)

68 out of 130 outlets have expected uplifts of more than 4 million IDR. These are ideal outlets to send promoters.

The net gain of sending promoters to those outlets is 477 million IDR.

In [ ]:
Net_gain = sum(selected_outlets) - len(selected_outlets)*promoters_threshold
Net_gain

### Next steps

1. The Test Group seems to be biased. A model must be built preferrably with an un-biased Test Group.
2. Play with different random splits of the train and test sets in all models to check how stable they are.
3. Causalml has two types of algorithms: A. Meta-learner algorithms like the one we are using here and B. Tree-based algorithms. Option B can be tested to see if it delivers better results. Some tests are made in the Appendix but with some technical issues to solve.

### APPENDIX: Other uplift models

#### Reassign column names to features dataframe X

In [ ]:
X.columns

In [ ]:
X_new_col_names = X
X_new_col_names.columns = ["f0", "f1", "f2", "f3", "f4", "f5", "f6", "f7", "f8", "f9",
                           "f10", "f11", "f12", "f13", "f14", "f15", "f16", "f17", "f18", "f19",
                           "f20", "f21", "f22", "f23", "f24", "f25", "f26", "f27", "f28", "f29",
                           "f30", "f31", "f32", "f33", "f34", "f35", "f36", "f37", "f38", "f39",
                           "f40", "f41", "f42", "f43", "f44", "f45", "f46", "f47", "f48", "f49"]
X_new_col_names.columns

#### Data split

In [ ]:
x_train, x_test, treatment_train, treatment_test, outcome_train, outcome_test = train_test_split(
        XX, treatment, y, test_size=0.2, random_state=42
    )

#### Meta-learner: t-learner

In [ ]:
from causalml.inference.meta import BaseTClassifier
from xgboost import XGBClassifier

In [ ]:
xgb_tlearner = BaseTClassifier(learner=XGBClassifier(random_state=42, n_estimators = 300,
                                    max_depth = 5, learning_rate = 0.1))

xgb_tlearner.fit(X=x_train, y=outcome_train, treatment=treatment_train)

In [ ]:
xgb_tlearner

In [ ]:
from causalml.metrics import plot

t_pred = xgb_tlearner.predict(X=x_test)

## Aggregating everything on a dataframe
valid_t = pd.DataFrame({'y': outcome_test,
                   'w': treatment_test,
                   'T-Learner': t_pred.reshape(-1), 
                  })

## Plotting the 3 types of uplift curve. 
plot(valid_t,kind='qini', outcome_col='y', treatment_col='w',figsize=(10, 3.3))

In [ ]:
from causalml.metrics import auuc_score, qini_score
print('AUUC:\n',auuc_score(valid_t))

#### Uplift trees

In [ ]:
from causalml.inference.tree import UpliftTreeClassifier
from causalml.inference.tree import uplift_tree_string, uplift_tree_plot

In [ ]:
uplift_tree = UpliftTreeClassifier(max_depth=5, min_samples_leaf=20, min_samples_treatment=5,
                                    n_reg=100, evaluationFunction='KL', control_name='control')

In [ ]:
uplift_tree.fit(x_train,np.where(treatment_train<1, "control", "treatment"), y=outcome_train)

#### Random uplift forest

In [ ]:
from causalml.inference.tree import UpliftRandomForestClassifier
from causalml.inference.tree import uplift_tree_string, uplift_tree_plot

In [ ]:
uplift_forest = UpliftRandomForestClassifier(control_name="control")

In [ ]:
uplift_forest.fit(x_train,
                  np.where(treatment_train<1, "control", "treatment"),
                  y=outcome_train)